Test 1 before better examples

In [1]:
import traceback

from pycram.bullet_world import BulletWorld, Object
from pycram.designators.object_designator import BelieveObject
from pycram.designators.action_designator import PickUpAction, PlaceAction
from pycram.pose import Pose
from pycram.process_module import simulated_robot
from pycram.enums import ObjectType

#Initialize the BulletWorld
world = BulletWorld()

#Define objects
kitchen = Object('kitchen', ObjectType.ENVIRONMENT, 'kitchen.urdf')
robot = Object('pr2', ObjectType.ROBOT, 'pr2.urdf')
cereal = Object('cereal', ObjectType.BREAKFAST_CEREAL, 'breakfast_cereal.stl', pose=Pose([1.4, 1, 0.95]))

#Create object designators
cereal_designator = BelieveObject(names=["cereal"])

#Define actions within the simulated_robot context manager
with simulated_robot:
    # Pick up the cereal
    pick_up_action = PickUpAction(object_designator_description=cereal_designator, arms=["right"], grasps=["grasp_type"]).resolve()
    pick_up_action.perform()

    # Calculate the target location for placing the cereal (3 steps to the right)
    target_location = Pose([cereal.get_position().position[0] + 0.6, cereal.get_position().position[1], cereal.get_position().position[2]])

    # Place the cereal at the target location
    place_action = PlaceAction(object_designator_description=cereal_designator, target_locations=[target_location], arms=["right"]).resolve()
    place_action.perform()

#Close the BulletWorld
world.exit()

pybullet build time: May 20 2022 19:44:17
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
[WARN] [1711545927.705700]: Failed to import Giskard messages
[WARN] [1711545927.709220]: Could not import RoboKudo messages, RoboKudo interface could not be initialized
Scalar element defined multiple times: limit
Scalar element defined multiple times: limit


KeyError: 'grasp_type'

In [1]:
from pycram.bullet_world import BulletWorld, Object
from pycram.process_module import simulated_robot
from pycram.designators.motion_designator import *
from pycram.designators.location_designator import *
from pycram.designators.action_designator import *
from pycram.designators.object_designator import *
from pycram.pose import Pose
from pycram.enums import ObjectType

world = BulletWorld()

#Objects
kitchen = Object("kitchen", ObjectType.ENVIRONMENT, "kitchen.urdf")
robot = Object("pr2", ObjectType.ROBOT, "pr2.urdf")
cereal = Object("cereal", ObjectType.BREAKFAST_CEREAL, "breakfast_cereal.stl", pose=Pose([1.4, 1, 0.95]))

#Object Designators
cereal_desig = ObjectDesignatorDescription(names=["cereal"])

with simulated_robot:
    # Initial actions
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    MoveTorsoAction([0.25]).resolve().perform()

    # Pick up the cereal
    pickup_pose = CostmapLocation(target=cereal_desig.resolve(), reachable_for=robot).resolve()
    pickup_arm = pickup_pose.reachable_arms[0]
    NavigateAction(target_locations=[pickup_pose.pose]).resolve().perform()
    PickUpAction(object_designator_description=cereal_desig, arms=[pickup_arm], grasps=["front"]).resolve().perform()

    # Move 3 steps to the right
    motion_designator = MotionDesignator(direction="right", distance=3)
    motion_designator.resolve().perform()

    # Place the cereal
    place_pose = CostmapLocation(target=cereal_desig.resolve(), reachable_for=robot).resolve()
    NavigateAction(target_locations=[place_pose.pose]).resolve().perform()
    PlaceAction(cereal_desig, target_locations=[place_pose.pose], arms=[pickup_arm]).resolve().perform()

    # Park the robot's arms again
    ParkArmsAction([Arms.BOTH]).resolve().perform()

world.exit()

pybullet build time: May 20 2022 19:44:17
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
[WARN] [1711617699.708743]: Failed to import Giskard messages
[WARN] [1711617699.713617]: Could not import RoboKudo messages, RoboKudo interface could not be initialized
Scalar element defined multiple times: limit
Scalar element defined multiple times: limit


AttributeError: 'Object' object has no attribute 'bullet_world_object'

In [11]:
import traceback
test = """from pycram.bullet_world import BulletWorld, Object
from pycram.process_module import simulated_robot
# Import Statements für Designatoren sind unvollständig.
from pycram.designators.motion_designator import *
from pycram.designators.location_designator import *
from pycram.designators.action_designator import *
from pycram.designators.object_designator import *
from pycram.enums import ObjectType, Arms
from pycram.pose import Pose

# Erstellung der BulletWorld
world = BulletWorld()

# Objekte hinzufügen
kitchen = Object('kitchen', ObjectType.ENVIRONMENT, 'kitchen.urdf')
robot = Object('pr2', ObjectType.ROBOT, 'pr2.urdf')
cereal = Object('cereal', ObjectType.BREAKFAST_CEREAL, 'breakfast_cereal.stl', pose=Pose([1.4, 1, 0.95]))

# Designatoren definieren
cereal_desig = ObjectDesignatorDescription(names=['cereal'])
robot_desig = ObjectDesignatorDescription(names=['pr2']).resolve()

# Simulierten Roboter aktivieren
with simulated_robot:
    # Roboteraktionen
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    MoveTorsoAction([0.25]).resolve().perform()

    # Position für das Aufnehmen der Cerealien bestimmen
    pickup_pose = CostmapLocation(target=cereal_desig.resolve(), reachable_for=robot_desig).resolve()
    pickup_arm = pickup_pose.reachable_arms[0]

    # Zum Cerealien navigieren und aufnehmen
    NavigateAction(target_locations=[pickup_pose.pose]).resolve().perform()
    PickUpAction(object_designator_description=cereal_desig, arms=[pickup_arm], grasps=['front']).resolve().perform()

    # Zielposition bestimmen und dorthin bewegen
    target_pose = Pose([pickup_pose.pose.position.x + 3, pickup_pose.pose.position.y, pickup_pose.pose.position.z], pickup_pose.pose.orientation)
    move_motion = MoveMotion(target=target_pose)
    move_motion.perform()

    # Cerealien auf der Kücheninsel ablegen
    place_island = SemanticCostmapLocation('kitchen_island_surface', kitchen.resolve(), cereal.resolve()).resolve()
    PlaceAction(object_designator_description=cereal_desig, target_locations=[place_island.pose], arms=[pickup_arm]).resolve().perform()

# Simulation beenden
world.exit()"""


test1 = """from pycram.bullet_world import BulletWorld, Object
from pycram.designators.action_designator import *
from pycram.designators.motion_designator import *
from pycram.designators.location_designator import *
from pycram.designators.object_designator import *
from pycram.process_module import simulated_robot
from pycram.pose import Pose
from pycram.enums import ObjectType, Arms
from pycram.process_module import simulated_robot
from pycram.pose import Pose
from pycram.enums import ObjectType, Arms
"""

try:
    # Attempt to execute the imports
    exec(test)
except Exception as e:
    print("---CODE IMPORT CHECK: FAILED---")
    # Catch any error during execution (e.g., ImportError, SyntaxError)
    error = f"Execution error: {traceback.format_exc()}"
    try:
        exec("world.exit()")
    except Exception as e:
        print("No World has been exited")
    print(error)

Scalar element defined multiple times: limit
Scalar element defined multiple times: limit


---CODE IMPORT CHECK: FAILED---
Execution error: Traceback (most recent call last):
  File "/tmp/ipykernel_15201/2632560316.py", line 66, in <module>
    exec(test)
  File "<string>", line 30, in <module>
  File "/home/julius/ros/ros_ws/src/pycram/src/pycram/designators/location_designator.py", line 144, in ground
    return next(iter(self))
  File "/home/julius/ros/ros_ws/src/pycram/src/pycram/designators/location_designator.py", line 198, in __iter__
    valid, arms = reachability_validator(maybe_pose, test_robot, target_pose,
  File "/home/julius/ros/ros_ws/src/pycram/src/pycram/pose_generator_and_validator.py", line 157, in reachability_validator
    resp = request_ik(target, robot, left_joints, left_gripper)
  File "/home/julius/ros/ros_ws/src/pycram/src/pycram/external_interfaces/ik.py", line 133, in request_ik
    inv = call_ik(base_link, end_effector, target_diff, robot, joints)
  File "/home/julius/ros/ros_ws/src/pycram/src/pycram/external_interfaces/ik.py", line 90, in call_i

In [1]:
test_string = """from pycram.bullet_world import BulletWorld, Object
from pycram.designators.action_designator import *
from pycram.designators.motion_designator import *
from pycram.designators.location_designator import *
from pycram.designators.object_designator import *
from pycram.process_module import simulated_robot
from pycram.pose import Pose
from pycram.enums import ObjectType, Arms"""
exec(test_string)

pybullet build time: Nov 28 2023 23:45:17
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
[WARN] [1712822782.572490]: Failed to import Giskard messages
[WARN] [1712822782.579082]: Could not import RoboKudo messages, RoboKudo interface could not be initialized


In [2]:
test_string = """from pycram.bullet_world import BulletWorld, Object
from pycram.designators.action_designator import *
from pycram.designators.motion_designator import *
from pycram.designators.location_designator import *
from pycram.designators.object_designator import *
from pycram.process_module import simulated_robot
from pycram.pose import Pose
from pycram.enums import ObjectType, Arms"""
try:
    # Attempt to execute the imports
    exec(test_string)
except Exception as e:
    print("---CODE IMPORT CHECK: FAILED---")
    # Catch any error during execution (e.g., ImportError, SyntaxError)
    error = f"Execution error: {e}"
    print(error)

In [12]:
from pycram.bullet_world import BulletWorld, Object
from pycram.designators.action_designator import *
from pycram.designators.motion_designator import *
from pycram.designators.location_designator import *
from pycram.designators.object_designator import *
from pycram.process_module import simulated_robot
from pycram.pose import Pose
from pycram.enums import ObjectType

# BulletWorld Definition
world = BulletWorld()

# Objects
robot = Object("pr2", ObjectType.ROBOT, "pr2.urdf", pose=Pose([1, 2, 0]))
apartment = Object("apartment", ObjectType.ENVIRONMENT, "apartment.urdf")
milk = Object("milk", ObjectType.MILK, "milk.stl", pose=Pose([2.5, 2, 1.02]), color=[1, 0, 0, 1])
cereal = Object("cereal", ObjectType.BREAKFAST_CEREAL, "breakfast_cereal.stl", pose=Pose([2.5, 2.3, 1.05]), color=[0, 1, 0, 1])
spoon = Object("spoon", ObjectType.SPOON, "spoon.stl", pose=Pose([2.4, 2.2, 0.85]), color=[0, 0, 1, 1])
bowl = Object("bowl", ObjectType.BOWL, "bowl.stl", pose=Pose([2.5, 2.2, 1.02]), color=[1, 1, 0, 1])

# Object Designators
robot_desig = BelieveObject(names=["pr2"]).resolve()
apartment_desig = BelieveObject(names=["apartment"]).resolve()
milk_desig = BelieveObject(names=["milk"]).resolve()
cereal_desig = BelieveObject(names=["cereal"]).resolve()
spoon_desig = BelieveObject(names=["spoon"]).resolve()
bowl_desig = BelieveObject(names=["bowl"]).resolve()

with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    MoveTorsoAction([0.25]).resolve().perform()
    
    # Navigate to the kitchen
    kitchen_location = SemanticCostmapLocation(urdf_link_name="kitchen_island_surface", part_of=apartment_desig, for_object=robot_desig).resolve()
    NavigateAction(target_locations=[kitchen_location.pose]).resolve().perform()
    
    # Open the drawer to get the spoon
    drawer_handle = ObjectPart(names=["sink_area_left_upper_drawer_main"], part_of=apartment_desig).resolve()
    OpenAction(drawer_handle, [Arms.RIGHT]).resolve().perform()
    PickUpAction(object_designator_description=spoon_desig, arms=[Arms.RIGHT], grasps=["front"]).resolve().perform()
    CloseAction(drawer_handle, [Arms.RIGHT]).resolve().perform()
    
    # Pick up the bowl
    PickUpAction(object_designator_description=bowl_desig, arms=[Arms.LEFT], grasps=["front"]).resolve().perform()
    
    # Navigate to the dining table
    dining_table_location = SemanticCostmapLocation(urdf_link_name="dining_table_surface", part_of=apartment_desig, for_object=bowl_desig).resolve()
    NavigateAction(target_locations=[dining_table_location.pose]).resolve().perform()
    
    # Place the bowl on the dining table
    PlaceAction(bowl_desig, target_locations=[dining_table_location.pose], arms=[Arms.LEFT]).resolve().perform()
    
    # Place the spoon on the dining table
    PlaceAction(spoon_desig, target_locations=[dining_table_location.pose], arms=[Arms.RIGHT]).resolve().perform()
    
    # Pick up the milk and cereal
    PickUpAction(object_designator_description=milk_desig, arms=[Arms.LEFT], grasps=["front"]).resolve().perform()
    PickUpAction(object_designator_description=cereal_desig, arms=[Arms.RIGHT], grasps=["front"]).resolve().perform()
    
    # Place the milk and cereal on the dining table
    PlaceAction(milk_desig, target_locations=[dining_table_location.pose], arms=[Arms.LEFT]).resolve().perform()
    PlaceAction(cereal_desig, target_locations=[dining_table_location.pose], arms=[Arms.RIGHT]).resolve().perform()

# Exit the BulletWorld
world.exit()

Unknown tag "material" in /robot[@name='apartment']/link[@name='coffe_machine']/collision[1]
Unknown tag "material" in /robot[@name='apartment']/link[@name='coffe_machine']/collision[1]


KeyError: 'kitchen_island_surface'

In [1]:
from pycram.designators.action_designator import *
from pycram.designators.location_designator import *
from pycram.designators.object_designator import *
from pycram.pose import Pose
from pycram.bullet_world import BulletWorld, Object
from pycram.process_module import simulated_robot
from pycram.enums import ObjectType

world = BulletWorld()
robot = Object("pr2", ObjectType.ROBOT, "pr2.urdf", pose=Pose([1, 2, 0]))
apartment = Object("apartment", ObjectType.ENVIRONMENT, "apartment.urdf")
milk = Object("milk", ObjectType.MILK, "milk.stl", pose=Pose([2.5, 2, 1.02]), color=[1, 0, 0, 1])
cereal = Object("cereal", ObjectType.BREAKFAST_CEREAL, "breakfast_cereal.stl", pose=Pose([2.5, 2.3, 1.05]), color=[0, 1, 0, 1])
spoon = Object("spoon", ObjectType.SPOON, "spoon.stl", pose=Pose([2.4, 2.2, 0.85]), color=[0, 0, 1, 1])
bowl = Object("bowl", ObjectType.BOWL, "bowl.stl", pose=Pose([2.5, 2.2, 1.02]), color=[1, 1, 0, 1])
apartment.attach(spoon, 'cabinet10_drawer_top')

robot_desig = BelieveObject(names=["pr2"])
apartment_desig = BelieveObject(names=["apartment"])
milk_desig = BelieveObject(names=["milk"])
cereal_desig = ObjectDesignatorDescription(names=["cereal"])
spoon_desig = ObjectDesignatorDescription(names=["spoon"])
bowl_desig = ObjectDesignatorDescription(names=["bowl"])

with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    MoveTorsoAction([0.25]).resolve().perform()

    # Navigate to and pick up the bowl
    bowl_location = CostmapLocation(target=bowl_desig.resolve, reachable_for=robot_desig).resolve()
    NavigateAction(target_locations=[bowl_location.pose]).resolve().perform()
    PickUpAction(object_designator_description=bowl_desig, arms=[bowl_location.reachable_arms[0]], grasps=["front"]).resolve().perform()

    # Navigate to and pick up the cereal
    cereal_location = CostmapLocation(target=cereal_desig.resolve(), reachable_for=robot_desig).resolve()
    NavigateAction(target_locations=[cereal_location.pose]).resolve().perform()
    PickUpAction(object_designator_description=cereal_desig, arms=[cereal_location.reachable_arms[0]], grasps=["front"]).resolve().perform()

    # Navigate to and pick up the milk
    milk_location = CostmapLocation(target=milk_desig.resolve(), reachable_for=robot_desig).resolve()
    NavigateAction(target_locations=[milk_location.pose]).resolve().perform()
    PickUpAction(object_designator_description=milk_desig, arms=[milk_location.reachable_arms[0]], grasps=["front"]).resolve().perform()

    # Navigate to and pick up the spoon from the drawer
    handle_desig = ObjectPart(names=["handle_cab10_t"], part_of=apartment_desig.resolve())
    drawer_open_loc = AccessingLocation(handle_desig=handle_desig.resolve(), robot_desig=robot_desig.resolve()).resolve()
    NavigateAction([drawer_open_loc.pose]).resolve().perform()
    OpenAction(object_designator_description=handle_desig, arms=[drawer_open_loc.arms[0]]).resolve().perform()
    spoon.detach(apartment)
    LookAtAction([apartment.get_link_pose("handle_cab10_t")]).resolve().perform()
    DetectAction(spoon_desig).resolve().perform()
    PickUpAction(spoon_desig, [drawer_open_loc.arms[0]], ["top"]).resolve().perform()

    # Place all items on the dining table
    dining_table_location = SemanticCostmapLocation(urdf_link_name="dining_table_surface", part_of=apartment_desig).resolve()
    NavigateAction([dining_table_location.pose]).resolve().perform()
    PlaceAction(bowl_desig, [dining_table_location.pose], [bowl_location.reachable_arms[0]]).resolve().perform()
    PlaceAction(cereal_desig, [dining_table_location.pose], [cereal_location.reachable_arms[0]]).resolve().perform()
    PlaceAction(milk_desig, [dining_table_location.pose], [milk_location.reachable_arms[0]]).resolve().perform()
    PlaceAction(spoon_desig, [dining_table_location.pose], [drawer_open_loc.arms[0]]).resolve().perform()

world.exit()

pybullet build time: Nov 28 2023 23:45:17
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
[WARN] [1714477238.836198]: Failed to import Giskard messages
[WARN] [1714477238.843108]: Could not import RoboKudo messages, RoboKudo interface could not be initialized
Unknown tag "material" in /robot[@name='apartment']/link[@name='coffe_machine']/collision[1]
Unknown tag "material" in /robot[@name='apartment']/link[@name='coffe_machine']/collision[1]


AttributeError: 'function' object has no attribute 'copy'

In [ ]:
from pycram.designators.action_designator import NavigateAction, LookAtAction, DetectAction, TransportAction, OpenAction, CloseAction
from pycram.designators.object_designator import BelieveObject, ObjectPart
from pycram.pose import Pose
from pycram.enums import Arms
from pycram.process_module import simulated_robot
from pycram.bullet_world import BulletWorld, Object
from pycram.enums import ObjectType
world = BulletWorld()
robot = Object("pr2", ObjectType.ROBOT, "pr2.urdf", pose=Pose([1, 2, 0]))
apartment = Object("apartment", ObjectType.ENVIRONMENT, "apartment.urdf")
milk = Object("milk", ObjectType.MILK, "milk.stl", pose=Pose([2.5, 2, 1.02]), color=[1, 0, 0, 1])
cereal = Object("cereal", ObjectType.BREAKFAST_CEREAL, "breakfast_cereal.stl", pose=Pose([2.5, 2.3, 1.05]), color=[0, 1, 0, 1])
spoon = Object("spoon", ObjectType.SPOON, "spoon.stl", pose=Pose([2.4, 2.2, 0.85]), color=[0, 0, 1, 1])
bowl = Object("bowl", ObjectType.BOWL, "bowl.stl", pose=Pose([2.5, 2.2, 1.02]), color=[1, 1, 0, 1])
apartment.attach(spoon, 'cabinet10_drawer_top')

robot_desig = BelieveObject(names=["pr2"])
apartment_desig = BelieveObject(names=["apartment"])

with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    MoveTorsoAction([0.25]).resolve().perform()

    # Navigate to the cereal
    NavigateAction(target_locations=[Pose([2.5, 2.3, 1.05])]).resolve().perform()

    # Pick up the cereal
    cereal_desig = DetectAction(BelieveObject(names=["cereal"])).resolve().perform()
    TransportAction(cereal_desig, ["right"], [Pose([2.8, 2.3, 1.05])]).resolve().perform()

    # Place the cereal three steps to the right
    TransportAction(cereal_desig, ["right"], [Pose([3.1, 2.3, 1.05])]).resolve().perform()

world.exit()


pybullet build time: Nov 28 2023 23:45:17
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


In [ ]:
from pycram.bullet_world import BulletWorld, Object
world = BulletWorld()

pybullet build time: Nov 28 2023 23:45:17
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


In [1]:
from pycram.bullet_world import BulletWorld, Object
from pycram.designators.location_designator import CostmapLocation, SemanticCostmapLocation
from pycram.designators.object_designator import ObjectDesignatorDescription
from pycram.designators.motion_designator import Arms, NavigateAction, ParkArmsAction, MoveTorsoAction, PickUpAction, PlaceAction
from pycram.process_module import simulated_robot
from pycram.pose import Pose
from pycram.enums import ObjectType
# Define the BulletWorld
world = BulletWorld()

# Define the objects
kitchen = Object('kitchen', ObjectType.ENVIRONMENT, 'kitchen.urdf')
robot = Object('pr2', ObjectType.ROBOT, 'pr2.urdf')
cereal = Object('cereal', ObjectType.BREAKFAST_CEREAL, 'breakfast_cereal.stl', pose=Pose([1.4, 1, 0.95]))

# Define the object designators
cereal_desig = ObjectDesignatorDescription(names=['cereal']).resolve()
robot_desig = ObjectDesignatorDescription(names=['pr2']).resolve()

# Actions and moves of the Robot
with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    MoveTorsoAction([0.25]).resolve().perform()

    # Navigate to the cereal
    pickup_pose = CostmapLocation(target=cereal_desig, reachable_for=robot_desig).resolve()
    NavigateAction(target_locations=[pickup_pose.pose]).resolve().perform()

    # Pick up the cereal
    pickup_arm = pickup_pose.reachable_arms[0]
    PickUpAction(object_designator_description=cereal_desig, arms=[pickup_arm], grasps=['front']).resolve().perform()

    # Calculate new pose three steps to the right
    new_pose = Pose([pickup_pose.pose.position.x + 3, pickup_pose.pose.position.y, pickup_pose.pose.position.z], pickup_pose.pose.orientation)

    # Place the cereal at the new location
    place_stand = CostmapLocation(new_pose, reachable_for=robot_desig, reachable_arm=pickup_arm).resolve()
    NavigateAction(target_locations=[place_stand.pose]).resolve().perform()
    PlaceAction(cereal_desig, target_locations=[place_stand.pose], arms=[pickup_arm]).resolve().perform()

# Close the BulletWorld
world.exit()

pybullet build time: Nov 28 2023 23:45:17
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
[WARN] [1715095580.541331]: Failed to import Giskard messages
[WARN] [1715095580.547716]: Could not import RoboKudo messages, RoboKudo interface could not be initialized


ImportError: cannot import name 'Arms' from 'pycram.designators.motion_designator' (/home/julius/ros/ros_ws/src/pycram/src/pycram/designators/motion_designator.py)

In [5]:
import sys
import os
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
import torch

# Eingabetext aus den Argumenten lesen
input_text = "Is there alien life."

login(token=".env")

# Llama3 laden
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
)

messages = [
    {"role": "system", "content": "You are a physicist."},
    {"role": "user", "content": input_text},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/julius/.cache/huggingface/token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


KeyboardInterrupt: 

In [5]:
import sys
import os
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from optimum.onnxruntime import ORTModelForCausalLM
from huggingface_hub import login
import torch
import accelerate

# Eingabetext aus den Argumenten lesen
input_text = "Is there alien life?"

login(token=".env")

# Llama3 laden
model_id = "meta-llama/Meta-Llama-3-8B"

nlp = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    tokenizer=tokenizer,
)

outputs = nlp(
    input_text
)
print(outputs[0]["generated_text"][len(input_text):])

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/julius/.cache/huggingface/token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The argument `from_transformers` is deprecated, and will be removed in optimum 2.0.  Use `export` instead


TypeError: ORTModelForCausalLM._from_transformers() got an unexpected keyword argument 'quantize'

In [ ]:
from pycram.bullet_world import BulletWorld, Object
from pycram.designators.location_designator import CostmapLocation, SemanticCostmapLocation
from pycram.designators.object_designator import BelieveObject, ObjectDesignatorDescription
from pycram.designators.motion_designator import NavigateAction, PickUpAction, PlaceAction, ParkArmsAction, MoveTorsoAction
from pycram.pose import Pose
from pycram.enums import ObjectType
# BulletWorld Definition
world = BulletWorld()

# Objects
kitchen = Object('kitchen', ObjectType.ENVIRONMENT, 'kitchen.urdf')
robot = Object('pr2', ObjectType.ROBOT, 'pr2.urdf')
cereal = Object('cereal', ObjectType.BREAKFAST_CEREAL, 'breakfast_cereal.stl', pose=Pose([1.4, 1, 0.95]))

# Object Designators
robot_desig = BelieveObject(names=['pr2']).resolve()
cereal_desig = BelieveObject(names=['cereal']).resolve()

# The 'with simulated_robot:'-Block (defines the Actions and moves of the Robot)
with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    MoveTorsoAction([0.25]).resolve().perform()

    # Navigate to the cereal
    NavigateAction(target_locations=[cereal.get_pose()]).resolve().perform()

    # Pick up the cereal
    pickup_arm = CostmapLocation(target=cereal_desig.resolve(), reachable_for=robot_desig).resolve().reachable_arms[0]
    PickUpAction(object_designator_description=cereal_desig, arms=[pickup_arm], grasps=['front']).resolve().perform()

    # Define the target location 3 steps to the right
    target_location = CostmapLocation(target=cereal_desig.resolve(), reachable_for=robot_desig)
    target_location.pose.position.x += 3

    # Navigate to the target location
    NavigateAction(target_locations=[target_location.pose]).resolve().perform()

    # Place the cereal at the target location
    PlaceAction(cereal_desig, target_locations=[target_location.pose], arms=[pickup_arm]).resolve().perform()

    # Park the arms again
    ParkArmsAction([Arms.BOTH]).resolve().perform()

# BulletWorld Close
world.exit()

In [2]:
from pycram.designators.action_designator import *
from pycram.designators.location_designator import *
from pycram.designators.object_designator import *
from pycram.pose import Pose
from pycram.bullet_world import BulletWorld, Object
from pycram.process_module import simulated_robot, with_simulated_robot
from pycram.enums import ObjectType
world = BulletWorld()
robot = Object('pr2', ObjectType.ROBOT, 'pr2.urdf')
kitchen = Object('kitchen', ObjectType.ENVIRONMENT, 'kitchen.urdf')
cereal = Object('cereal', ObjectType.BREAKFAST_CEREAL, 'breakfast_cereal.stl', pose=Pose([1.4, 1, 0.95]))

robot_desig = ObjectDesignatorDescription(names=['pr2']).resolve()
cereal_desig = ObjectDesignatorDescription(names=['cereal'])
target_pose = Pose([1.4, 4, 0.95])  # 3 steps to the right
target_location = CostmapLocation(target=cereal_desig.resolve(), reachable_for=robot_desig)

@with_simulated_robot
def move_and_detect(obj_type):
    NavigateAction(target_locations=[Pose([1.4, 1, 0.95])]).resolve().perform()
    LookAtAction(targets=[Pose([1.4, 1, 0.95])]).resolve().perform()
    object_desig = DetectAction(ObjectDesignatorDescription(types=[obj_type])).resolve().perform()
    return object_desig

try:
    with simulated_robot:
        ParkArmsAction([Arms.BOTH]).resolve().perform()
        MoveTorsoAction([0.25]).resolve().perform()
    
        cereal_desig = move_and_detect(ObjectType.BREAKFAST_CEREAL)
    
        pickup_arm = target_location.reachable_arms[0]
        PickUpAction(object_designator_description=cereal_desig, arms=[pickup_arm], grasps=['front']).resolve().perform()
    
        place_location = CostmapLocation(target_pose, reachable_for=robot_desig, reachable_arm=pickup_arm)
        NavigateAction(target_locations=[place_location.pose]).resolve().perform()
    
        PlaceAction(cereal_desig, target_locations=[target_pose], arms=[pickup_arm]).resolve().perform()
    
        ParkArmsAction([Arms.BOTH]).resolve().perform()

except Exception as e:
    print(e, type(e))
    
world.exit()

Scalar element defined multiple times: limit
Scalar element defined multiple times: limit


Could not find an object with the type ObjectType.BREAKFAST_CEREAL in the FOV of the robot <class 'pycram.plan_failures.PerceptionObjectNotFound'>
